In [105]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Reshape
from sklearn.utils import resample

In [106]:
# Carregando o dataset
data = pd.read_csv('C:/Users/mrsilva/Pessoal/Senac/creditcard_2023.csv')

# Contando a quantidade de transações fraudulentas
num_frauds = data['Class'].sum()
print(f"Número de transações fraudulentas: {num_frauds}")

Número de transações fraudulentas: 284315


In [107]:
# Separando features e target
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']


In [108]:
# Dividindo o conjunto de dados em treino, validação e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [109]:
# Separar as classes majoritária e minoritária
major_class = data[data['Class'] == 0]
minor_class = data[data['Class'] == 1]

# Fazer oversampling da classe minoritária para igualar à classe majoritária
minor_oversampled = resample(minor_class, replace=True, n_samples=len(major_class), random_state=42)

# Combinar as classes reamostradas com a classe majoritária
balanced_data = pd.concat([major_class, minor_oversampled])


In [110]:
# Separando novamente os dados balanceados em features e target
X_balanced = balanced_data.drop(['Class', 'id'], axis=1)
y_balanced = balanced_data['Class']

In [111]:
# Dividindo os dados balanceados em treino e teste
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Dividindo o conjunto de treinamento balanceado em treinamento final e validação
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2, random_state=42)


In [112]:
# Modelo Convolutional Neural Network (CNN)
from tensorflow.keras.layers import Dropout

def build_cnn_model_with_dropout():
    input_layer = Input(shape=(X_train_final.shape[1],))  
    x = Reshape((X_train_final.shape[1], 1))(input_layer) 
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.25)(x)  # Exemplo de adição de Dropout após a camada de pooling
    x = Flatten()(x)
    x = Dropout(0.5)(x)  # Adicionando Dropout antes da camada densa
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [113]:
# Preparando dados balanceados para a CNN com novo input_shape
X_train_final_cnn = X_train_final.values.reshape((X_train_final.shape[0], X_train_final.shape[1], 1))
X_val_cnn = X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_balanced_cnn = X_test_balanced.values.reshape((X_test_balanced.shape[0], X_test_balanced.shape[1], 1))


In [114]:
# Treinando o modelo com o conjunto final de treinamento e validação
cnn_model = build_cnn_model_with_dropout()
history = cnn_model.fit(X_train_final_cnn, y_train_final, epochs=10, batch_size=64, validation_data=(X_val_cnn, y_val))


Epoch 1/10
5687/5687 [==============================] - 57s 10ms/step - loss: 0.1203 - accuracy: 0.9565 - val_loss: 0.0842 - val_accuracy: 0.9669
Epoch 2/10
5687/5687 [==============================] - 51s 9ms/step - loss: 0.0991 - accuracy: 0.9639 - val_loss: 0.0782 - val_accuracy: 0.9703
Epoch 3/10
5687/5687 [==============================] - 56s 10ms/step - loss: 0.0960 - accuracy: 0.9657 - val_loss: 0.0759 - val_accuracy: 0.9721
Epoch 4/10
5687/5687 [==============================] - 55s 10ms/step - loss: 0.0957 - accuracy: 0.9659 - val_loss: 0.0745 - val_accuracy: 0.9734
Epoch 5/10
5687/5687 [==============================] - 54s 10ms/step - loss: 0.0940 - accuracy: 0.9666 - val_loss: 0.0740 - val_accuracy: 0.9722
Epoch 6/10
5687/5687 [==============================] - 51s 9ms/step - loss: 0.0942 - accuracy: 0.9666 - val_loss: 0.0745 - val_accuracy: 0.9717
Epoch 7/10
5687/5687 [==============================] - 53s 9ms/step - loss: 0.0943 - accuracy: 0.9666 - val_loss: 0.0745 - va

In [ ]:
cnn_model.save('CNN_fraude_cartao.h5')

In [115]:
# Realizando previsões com a CNN nos dados de validação
cnn_predictions_val = cnn_model.predict(X_val_cnn)
cnn_predictions_val = np.round(cnn_predictions_val)

2844/2844 [==============================] - 5s 2ms/step


In [116]:
# Avaliando o desempenho da CNN nos dados de validação
cnn_accuracy_val = accuracy_score(y_val, cnn_predictions_val)
cnn_confusion_val = confusion_matrix(y_val, cnn_predictions_val)
cnn_precision_val = precision_score(y_val, cnn_predictions_val)
cnn_recall_val = recall_score(y_val, cnn_predictions_val)
cnn_f1_val = f1_score(y_val, cnn_predictions_val)

In [117]:
print("\n------CNN Metrics on Validation Data:---------\n")
print(f"Accuracy: {cnn_accuracy_val}\n")
print(f"Confusion Matrix:\n{cnn_confusion_val}")
print(f"Precision: {cnn_precision_val}\n")
print(f"Recall: {cnn_recall_val}\n")
print(f"F1 Score: {cnn_f1_val}")


------CNN Metrics on Validation Data:---------

Accuracy: 0.9730383266835932

Confusion Matrix:
[[45041   626]
 [ 1827 43487]]
Precision: 0.9858091718994401

Recall: 0.9596813346868518

F1 Score: 0.972569805539714
